## MNIST Classification using PyTorch 

In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets

#### Loading the Data

In [2]:
train = datasets.MNIST('', train=True,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))


test = datasets.MNIST('', train=False,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

100.1%

Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw


113.5%

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw


100.4%

Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw


180.4%

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw
Processing...
Done!


In [4]:
trainset = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [21]:
img_shape = train.data[0].shape[0]
img_shape

28

In [25]:
num_classes = len(train.classes)
num_classes

10

#### Building the Neural Network

In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [52]:
class Network(nn.Module):
    def __init__(self, img_shape=28, num_classes=10):
        super().__init__()
        
        self.img_shape = img_shape
        self.dense1_op = 64
        self.dense2_op = 64
        self.dense3_op = 64
        self.num_classes = num_classes
        
        
        self.dense1 = nn.Linear(img_shape * img_shape, self.dense1_op)
        self.dense2 = nn.Linear(self.dense1_op, self.dense2_op)
        self.dense3 = nn.Linear(self.dense2_op, self.dense3_op)
        self.dense4 = nn.Linear(self.dense3_op, self.num_classes)
        
    
    ## forward pass
    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        x = self.dense4(x)
        
        return F.log_softmax(x, dim=1)

In [53]:
net = Network(img_shape=img_shape, num_classes=num_classes)

In [54]:
net

Network(
  (dense1): Linear(in_features=784, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=64, bias=True)
  (dense3): Linear(in_features=64, out_features=64, bias=True)
  (dense4): Linear(in_features=64, out_features=10, bias=True)
)

#### Training the Network

In [55]:
from torch.optim import Adam

In [56]:
optimizer = Adam(net.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [57]:
epochs = 10

In [58]:
for epoch in range(epochs):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1, img_shape*img_shape))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print('Loss: ', loss)

Loss:  tensor(0.0594, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2667, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0650, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0986, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1532, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0098, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0217, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0712, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0529, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0067, grad_fn=<NllLossBackward>)


#### Train Accuracy

In [61]:
correct = 0
total = 0

with torch.no_grad():
    for data in trainset:
        X, y = data
        output = net(X.view(-1, img_shape*img_shape))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print('Accuracy: ', round(correct/total, 3) * 100)

Accuracy:  98.7


#### Test Accuracy

In [62]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, img_shape*img_shape))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print('Accuracy: ', round(correct/total, 3) * 100)

Accuracy:  97.2


#### Credits to SentDex on Youtube - https://www.youtube.com/channel/UCfzlCWGWYyIQ0aLC5w48gBQ